In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
drive.mount('/content/drive')

your_google_drive_folder = 'dogs3'
data_folder = os.path.join('/content/drive/MyDrive/Dogs2', your_google_drive_folder)


img_height, img_width = 150, 150
batch_size = 32
num_classes = 5
epochs = 10

data_folder = '/content/drive/MyDrive/Dogs2'


base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

for layer in base_model.layers:
    layer.trainable = False

model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(1000, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1000, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(num_classes, activation='softmax'))

model.compile(optimizer=optimizers.Adam(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

def lr_scheduler(epoch, lr):
    if epoch % 5 == 0 and epoch != 0:
        return lr * 0.9
    return lr

lr_schedule = LearningRateScheduler(lr_scheduler)

early_stopping = EarlyStopping(monitor='val_accuracy', patience=1000, restore_best_weights=True)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_folder,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_folder,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    epochs=700,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32,
    callbacks=[lr_schedule, early_stopping]
)


test_loss, test_acc = model.evaluate(validation_generator)
print(f'Test accuracy: {test_acc}')


model.save('/content/drive/MyDrive/fine_tuned_dog_breed_prediction_model_mobilenetv2.h5')


Mounted at /content/drive
9406464/9406464 [==============================] - 0s 0us/step


Found 142 images belonging to 5 classes.
Found 32 images belonging to 5 classes.
Epoch 1/500
4/4 [==============================] - 63s 19s/step - loss: 1.5924 - accuracy: 0.5000 - val_loss: 0.4915 - val_accuracy: 0.8125 - lr: 0.0010
Epoch 2/500
4/4 [==============================] - 5s 1s/step - loss: 0.6571 - accuracy: 0.8182 - val_loss: 0.7700 - val_accuracy: 0.7188 - lr: 0.0010
Epoch 3/500
4/4 [==============================] - 6s 1s/step - loss: 0.4128 - accuracy: 0.8828 - val_loss: 1.2646 - val_accuracy: 0.7188 - lr: 0.0010
Epoch 4/500
4/4 [==============================] - 5s 1s/step - loss: 0.5568 - accuracy: 0.8909 - val_loss: 1.7225 - val_accuracy: 0.6562 - lr: 0.0010
Epoch 5/500
4/4 [==============================] - 7s 2s/step - loss: 0.2342 - accuracy: 0.9364 - val_loss: 2.4727 - val_accuracy: 0.6875 - lr: 0.0010
Epoch 6/500
4/4 [==============================] - 5s 1s/step - loss: 0.4912 - accuracy: 0.8906 - val_loss: 2.9796 - val_accuracy: 0.6250 - lr: 9.0000e-04
Epoch 7

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
